# Tutorial

## Compilation of FEASST

The compilation of FEASST is described in the README of the base directory of FEASST.
Or visit the website at https://pages.nist.gov/feasst
These tutorials feature the text-based interface of FEASST.
There is also a C++ and Python interface.

## Canonical ensemble Lennard-Jones Monte Carlo

In this example, we will reproduce the average energy reported in the NIST SRSW https://mmlapps.nist.gov/srs/LJ_PURE/mc.htm at T*=0.9 and $\rho*=0.009$ using a Monte Carlo simulation of a bulk Lennard Jones fluid.
The simulation will be initialized with a desired number of particles in a cubic box, equilibrated while tuning the maximum trial displacement and then simulated to obtain the ensemble average energy.

To begin, a Python string, `script`, is used to write the file, `script.txt`, which is then run by the ```fst``` executable.
Within `script`, a [MonteCarlo](../plugin/monte_carlo/README.rst) object is created.
Then, a random number generator is given a seed.
The available arguments of [RandomMT19937](../plugin/math/doc/RandomMT19937.rst) can be found in the documentation of that object, or its base-class, [Random](../plugin/math/doc/Random.rst). 
Arguments of objects are separated by a space and are provided first as the name of the argument and then the value. In this case, `seed` is the argument name and `1572362164` is the argument value. 

In [1]:
script="""
# comments begin with the '#' symbol
MonteCarlo
RandomMT19937 seed 1572362164
"""

def run(script):
    with open('script.txt', 'w') as file: file.write(script)
    import subprocess
    syscode = subprocess.call("~/feasst/build/bin/fst < script.txt > script.log", shell=True, executable='/bin/bash')
    with open('script.log', 'r') as file: print(file.read(), '\n', 'exit:', syscode)
run(script)

# FEASST version: v0.16.0-98-gb599f38530-hwh/tutorial_update
RandomMT19937 seed 1572362164  
# initializing random number generator with seed: 1572362164
 
 exit: 0


The output of ```fst``` in ```script.log``` is displayed. It includes the FEASST version as well as a confirmation of the seed. Note that the version shown may not match your version.

If there is a typo in the arguments, this may result in an exception which will print to the terminal. Let's test this now by changing the `seed` argument to a non-existent argument such as `sd`. Because the argument `sd` is not recognized by [RandomMT19937](../plugin/math/doc/RandomMT19937.rst), a nonzero system code is returned and a verbose error should appear in your terminal.

In [2]:
run(script+"""RandomMT19937 sd 123""")

terminate called after throwing an instance of 'feasst::CustomException'
  what():  Throw on proc 0 : # Assertion `args.size() == 1 && args.begin()->first.empty() && args.begin()->second.empty()` failed0 plugin/utils/src/arguments.cpp:112: unused argument(s): sd 123 . If the arguments are unused then that means the objects did not expect the first keyword supplied in the above argument pair(s). Thus, there was likely a typo or the keyword is intended for a different class.


# FEASST version: v0.16.0-98-gb599f38530-hwh/tutorial_update
RandomMT19937 seed 1572362164  
# initializing random number generator with seed: 1572362164
RandomMT19937 sd 123  
 
 exit: 134


/bin/bash: line 1: 10410 Aborted                 (core dumped) ~/feasst/build/bin/fst < script.txt > script.log


The next step is to add a [Configuration](../plugin/configuration/README.rst).
In this example, a simple cubic periodic box is defined based on the number of particles and density reported by Frenkel and Smit.
Because the cubic length needs to be input to high precision, a Python format string is used to input a variable enclosed in curly brackets.

In addition, the first particle type with an index of `0` is defined by the file `lj.fstprt`.
Any argument value beginning with "/feasst" will have that beginning replaced with [feasst::install_dir()](../plugin/utils/doc/io.rst), and thus the full path to the file `lj.fstprt` is displayed in `script.log` shown below.
See [Forcefield](../forcefield/README.rst) for more information about the format of the data file, which is a LAMMPS-inspired file with some major differences.

In [3]:
num_particles=500
density=0.009
script+="""
Configuration cubic_box_length {length} particle_type0 /feasst/forcefield/lj.fstprt
""".format(length=(num_particles/density)**(1./3.))
run(script)

# FEASST version: v0.16.0-98-gb599f38530-hwh/tutorial_update
RandomMT19937 seed 1572362164  
# initializing random number generator with seed: 1572362164
Configuration cubic_box_length 38.15714141844439 particle_type0 /home/hwh/feasst/forcefield/lj.fstprt  
 
 exit: 0


Two [Potentials](../plugin/system/doc/Potential.rst) are introduced.
First, the pair-wise [Lennard-Jones (LJ) model](../plugin/system/doc/LennardJones.rst).
Second, [long-range corrections](../plugin/system/doc/LongRangeCorrections.rst), which approximately account for the cut off of the LJ potential by assuming a pair-wise radial distance distribution function of unity.

In [4]:
script+="""
Potential Model LennardJones
Potential VisitModel LongRangeCorrections
"""

Next, set [ThermoParams](../plugin/system/doc/ThermoParams.rst), such as temperature and chemical potential of each particle type.
The initial configuration will be generated with grand canonical particle additions, so a chemical potential is included, but will not contribute to canonical ensemble production simulations.

In addition, [Metropolis](../plugin/monte_carlo/doc/Metropolis.rst) acceptance criteria is utilized.

In [5]:
script+="""
ThermoParams beta {beta} chemical_potential0 -1
Metropolis
""".format(beta=1./0.9)

A [TrialTranslate](../plugin/monte_carlo/doc/TrialTranslate.rst) is then introduced which attempts to translate a random particle by a random distance which is bound in each dimension by a `tunable_param`.
This parameter may be adjusted to obtain a desired acceptance ratio, `tunable_target_acceptance`, with the help of [Tune](../plugin/steppers/doc/Tune.rst).
The line `set_variable name value` will replace any use of `name` in subsequent values, as shown in `script.log`.

In [6]:
script+="""
TrialTranslate weight 1 tunable_param 2 tunable_target_acceptance 0.2
set_variable trials_per 1e4
Tune trials_per trials_per
"""
run(script)

# FEASST version: v0.16.0-98-gb599f38530-hwh/tutorial_update
RandomMT19937 seed 1572362164  
# initializing random number generator with seed: 1572362164
Configuration cubic_box_length 38.15714141844439 particle_type0 /home/hwh/feasst/forcefield/lj.fstprt  
Potential Model LennardJones  
Potential VisitModel LongRangeCorrections  
ThermoParams beta 1.1111111111111112 chemical_potential0 -1  
Metropolis  
TrialTranslate tunable_param 2 tunable_target_acceptance 0.2 weight 1  
Tune trials_per 1e4  
 
 exit: 0


[CheckEnergy](../plugin/steppers/doc/CheckEnergy.rst) asserts that the optimized energy calculations match the unoptimized calculations within a tolerance.
Because the unoptimized energy calculation can be expensive, this check is only performed every `trials_per`, as defined above.

In [7]:
script+="""
CheckEnergy trials_per trials_per tolerance 1e-8
"""

With the help of [TrialAdd](../plugin/monte_carlo/doc/TrialTranslate.rst), we can now generate an initial configuration with a [Run](../plugin/monte_carlo/doc/Run.rst) until the desired number of particles are reached. [RemoveTrial](../plugin/monte_carlo/doc/Run.rst) is then used to simulate the Canonical ensemble.

In [8]:
script+="""
TrialAdd weight 2 particle_type 0
Run until_num_particles {num_particles}
RemoveTrial name TrialAdd
""".format(num_particles=num_particles)
run(script)

# FEASST version: v0.16.0-98-gb599f38530-hwh/tutorial_update
RandomMT19937 seed 1572362164  
# initializing random number generator with seed: 1572362164
Configuration cubic_box_length 38.15714141844439 particle_type0 /home/hwh/feasst/forcefield/lj.fstprt  
Potential Model LennardJones  
Potential VisitModel LongRangeCorrections  
ThermoParams beta 1.1111111111111112 chemical_potential0 -1  
Metropolis  
TrialTranslate tunable_param 2 tunable_target_acceptance 0.2 weight 1  
Tune trials_per 1e4  
CheckEnergy tolerance 1e-8 trials_per 1e4  
TrialAdd particle_type 0 weight 2  
Run until_num_particles 500  
RemoveTrial name TrialAdd  
 
 exit: 0


The simulation is then [Run](../plugin/monte_carlo/doc/Run.rst) for a number of trials before [RemoveModify](../plugin/monte_carlo/doc/Run.rst) disabled tuning of the maximum displacement.

In [9]:
script+="""
Run num_trials 1e5
RemoveModify name Tune
"""

Finally, it is time to obtain the ensemble average energy from a production simulation.
First, a [Log](../plugin/steppers/doc/Log.rst) periodically outputs the instantaneous status of the trials, but it is more useful for watching the progress of a simulation than actually calculating quantities.
Instead, [Energy](../plugin/steppers/doc/Energy.rst) may be used to precisely compute the average energy by accumulating its value after every trial.
A trajectory in the XYZ format may also be produced using [Movie](..plugin/steppers/doc/Movie.rst).


Additional [Analyze](../plugin/monte_carlo/doc/Analyze.rst) or [Modify](../plugin/monte_carlo/doc/Modify.rst) may be added.
First, 

to perform some task contingent upon the number of attempted trials.

In [10]:
script+="""
Log trials_per trials_per file_name lj.txt
Energy trials_per_write trials_per file_name lj_en.txt
Movie trials_per trials_per file_name lj.xyz
Run num_trials 1e6
"""
run(script)

# FEASST version: v0.16.0-98-gb599f38530-hwh/tutorial_update
RandomMT19937 seed 1572362164  
# initializing random number generator with seed: 1572362164
Configuration cubic_box_length 38.15714141844439 particle_type0 /home/hwh/feasst/forcefield/lj.fstprt  
Potential Model LennardJones  
Potential VisitModel LongRangeCorrections  
ThermoParams beta 1.1111111111111112 chemical_potential0 -1  
Metropolis  
TrialTranslate tunable_param 2 tunable_target_acceptance 0.2 weight 1  
Tune trials_per 1e4  
CheckEnergy tolerance 1e-8 trials_per 1e4  
TrialAdd particle_type 0 weight 2  
Run until_num_particles 500  
RemoveTrial name TrialAdd  
Run num_trials 1e5  
RemoveModify name Tune  
Log file_name lj.txt trials_per 1e4  
Energy file_name lj_en.txt trials_per_write 1e4  
Movie file_name lj.xyz trials_per 1e4  
Run num_trials 1e6  
 
 exit: 0


Compare the average energy to the NIST SRSW https://mmlapps.nist.gov/srs/LJ_PURE/mc.htm?

In [12]:
import pandas as pd
df=pd.read_csv('lj_en.txt')
print('U/N=', df['average'][0]/num_particles, '+/-', df['block_stdev'][0]/num_particles)
import math
assert(abs(-8.9936E-02-df['average'][0]/num_particles) < 3*math.sqrt(2.44E-05**2+(df['block_stdev'][0]/num_particles)**2))

U/N= -0.09026566972510794 +/- 0.000485748099308276


You should also find the `lj.xyz` trajectory file with an automatically-generated `lj.xyz.vmd` file for use with VMD (e.g., `vmd -e lj.xyz.vmd`).

Did this tutorial work as expected? Did you find any inconsistencies or have any comments? Please [contact](../CONTACT.rst) us. Any feedback is appreciated!

## Additional tutorials

After completing this basic tutorial, check out the tutorials specific to each [Plugin](../plugin/README.rst).
For example, see the tutorials of the [System](../plugin/system/README.rst), [MonteCarlo](../plugin/monte_carlo/README.rst) and [flat histogram](../plugin/flat_histogram/README.rst) plugins.